# Russian 2011–2021 budget cash flow analysis | Creating the dataset

In this notebook, I explain each step of the creation of the regional budgets dataset, which contains data from several Russian official sources, namely, the Federal Treasury, the Ministry of Finance, the Federal Statistics Service, and the Central Bank.

The execution of the whole notebook takes about 15 minutes.

## Table of Contents <a class="anchor" id="contents"></a>

* [Initial Data](#initial_data)
* [Dataset 1: Regional Budgets](#regional_budgets)
    * [Extracting the data](#extracting_the_desired_data)
    * [Concatinating 1,870 dataframes](#making_two_dataframes)
    * [Tidying the data](#tidying_the_data)
* [Datasets 2-5: Population, Earnings, Poverty, USDRUB](#datasets_2_5)
    * [Transforming and appending the data](#transforming_the_data)
* [Dataset 6: Taxes](#dataset_6)
    * [The assignment of codes to taxes](#codes_to_taxes)
    * [Extracting regions' names](#extracting_names)
    * [Joining taxes to budgets](#joining_taxes)
* [Checking for mismatches in data](#mismatches)
    * [Checking for "disappeared" taxes](#disappeared)
    * [Checking for discrepancies in sums of taxes](#discrepancies)
* [Dataset 7: Federal Budget](#dataset_7)
* [Codification](#codification)
* [Creating a dataset for analysis](#creating_the_dataset)

## [Initial Data](#contents) <a class="anchor" id="initial_data"></a>

#### To construct the database for analysis, we'll synthesize several independent datasets:

| <font size = '3'>Data</font> | <font size = '3'>Source</font> | <font size = '3'>Description/Usage</font> |
| :--- | :--- | :--- |
| <font size = '2'><b>Consolidated regional budgets' execution</b></font> | <font size = '2'>Federal Treasury of Russia</font> | <font size = '2'>For each year between 2011 and 2021, the Treasury has published a dataset containing 85 XLS files, each containing a dataframe on budget revenues and a dataframe on budget spending.</font> | 
| <font size = '2'><b>Federal budget execution</b></font> | <font size = '2'>Federal Treasury of Russia</font> | <font size = '2'>The federal budget dataset contains 11 XLS files, each with a dataframe for revenues and expenditures.</font> |
| <font size = '2'><b>Taxes</b></font> | <font size = '2'>Ministry of Finance</font> | <font size = '2'>Almost all VAT and mining taxes collected in regions go directly to the federal budget and are absent in the Treasury data on regions. We need data from the Ministry of Finance to estimate the total amount of collected taxes.</font> |
| <font size = '2'><b>Population</b></font> | <font size = '2'>Federal Statistics Service</font> | <font size = '2'>The population data is needed to count budget spending per capita.</font> |
| <font size = '2'><b>Real income, per capita income, poverty level</b></font> | <font size = '2'>Federal Statistics Service</font>  | <font size = '2'>We need them to estimate the changes in the regions' dwellers' wealth.</font> |
| <font size = '2'><b>USDRUB exchange rates</b></font> | <font size = '2'>Bank of Russia</font> | <font size = '2'>Will also be used for estimating per capita spending.</font> |

In [1]:
import pandas as pd
import numpy as np
from numpy.core.defchararray import find

import glob
import os

from time import time
from time import perf_counter

import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

## [Dataset 1: Regional Budgets](#contents) <a class="anchor" id="regional_budgets"></a>

For each year, there's a Treasury database, containing an XLS file for each of the 85 Russian regions, with separate sheets for the revenues and for spending. In this chunk, we'll collect this data into a single dictionary with a year (2011–2021), a region code (1–85), and type of money flow (revenues, spending) as keys. A loop will turn to each year's folder, read all the XLS files in it, find the pages we are interested in, and append them to the dictionary (x for revenue, y for spending).

The execution will take about 6.7 minutes.

In [2]:
time1 = time()
start = perf_counter()

year_dirs = glob.glob(r"./budget_data/reg/reg_20??") # reading all the regional folders for 2000s from the directory

budget_dict = dict()

for year_dir in year_dirs:
    year = int(year_dir[-4:])  # getting the year from the folder name

    budget_dict[year] = []

    files = glob.glob(os.path.join(
        year_dir, "*.xls"))  # reading all XLS files from the folder

    # Typically, the regional budget XLS file has four tabs, with the first one for revenues and the second for spending.
    # But older files (up to 2016) also have hidden sheets with metadata as the first sheets, so we'll index them separately
    # to ignore these sheets.

    if year >= 2017:
        for f in files:
            x = pd.read_excel(f)
            x['filename'] = f  # filenames are needed to extract unique region numbers
            y = pd.read_excel(f, sheet_name=1)
            y['filename'] = f
            budget_dict[year].append([x, y])
    else:
        for f in files:
            x = pd.read_excel(f, sheet_name=1)
            x['filename'] = f
            y = pd.read_excel(f, sheet_name=2)
            y['filename'] = f
            budget_dict[year].append([x, y])

stop = perf_counter()
print('minutes passed:', (stop - start) / 60)

minutes passed: 7.09762585


Here's what one of our data frames looks like, and how we load it: by year, region number, and money flow type (income or spending):

In [3]:
budget_dict[2015][19][1].head(
)  # 2015 represents the year, 19 represents the number of the region in the Treasury's database,
# and 1 represents money flow type (spending)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Форма 0503317 с. 2,filename
0,NaN,2. Расходы бюджета,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./budget_data/reg\reg_2015\20-2116914.XLS
1,Наименование показателя,Код строки,Код расхода по бюджетной классификации,NaN,NaN,NaN,NaN,Утвержденные бюджетные назначения,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,./budget_data/reg\reg_2015\20-2116914.XLS
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,консолидированный бюджет субъекта Российской Ф...,"суммы, подлежащие исключению в рамках консолид...",консолидированный бюджет субъекта Российской Ф...,...,"суммы, подлежащие исключению в рамках консолид...",консолидированный бюджет субъекта Российской Ф...,"суммы, подлежащие исключению в рамках консолид...",бюджет субъекта Российской Федерации,бюджеты внутригородских муниципальных образова...,бюджеты городских округов,бюджеты муниципальных районов,бюджеты городских и сельских поселений,бюджет территориального государственного внебю...,./budget_data/reg\reg_2015\20-2116914.XLS
3,1,2,3,NaN,NaN,NaN,NaN,4,5,6,...,15,16,17,18,19,20,21,22,23,./budget_data/reg\reg_2015\20-2116914.XLS
4,Расходы бюджета - ИТОГО,200,NaN,NaN,x,NaN,NaN,139308638783.820007,25000000,116289012291.389999,...,22000000,108127577166.339996,18566323966.529999,85653406115.190002,0,28740274839.57,10408051149.93,1892169028.18,22838507518.57,./budget_data/reg\reg_2015\20-2116914.XLS


### [Extracting the data](#contents) <a class="anchor" id="extracting_the_desired_data"></a>

<b>The dataframes in the dictionary are quite a mess so far, so let's make them readable.</b> For the analysis, we need to reduce each dataframe to only four columns:

- The first column contains the name of the budget item (personal income tax, property tax, etc.).

- The column with the unique budget item code makes it convenient to index the dataframe.

- One of the transaction amount columns. There are several levels of budget transactions: the region solely; the region including all of its settlements; and the region including all of its settlements as well as pension, state medical, and social insurance funds. For each, there are two columns: one for the planned amount of money and one for the actual amount. For our analysis, we'll use the data on the actual execution of the "consolidated budget of the region and all of its settlements".

- The last column that has been created in the previous chunk contains the filename with a unique region code.
 

In the next chunk, we create a loop that will iterate over 1,870 datasets in the dictionary, find the columns we are interested in, and replace the old dataframe with the new 4-column dataframe.

The execution is about 9 minutes.

In [4]:
# In revenue data frames, I locate columns by unique substrings;
# In spending data frames, I locate columns by unique substrings or by their relative 
# position to the spending name column.

start = perf_counter()

years = pd.Series(range(2011, 2022))

for year in years:

    for index in range(len(budget_dict[year])):

        inc = budget_dict[year][index][0].applymap(lambda s: s.lower()
                                                   if type(s) == str else s)
        spnd = budget_dict[year][index][1].applymap(lambda s: s.lower()
                                                    if type(s) == str else s)

        # REVENUES

        # fixing some mess in the frames for the 2016 to make all the columns findable:
        inc = inc.replace('-', '', regex=True)
        inc = inc.replace('\n', '', regex=True)
        inc = inc.replace('ро ванный', 'рованный', regex=True)

        # locating the columns by the unique substrings:
        inc['revenue_type_rus'] = inc[inc.columns[(find(
            inc.to_numpy().astype(str), 'в том числе') >= 0).any(0)][0]]
        inc['revenue'] = inc[inc.columns[(find(
            inc.to_numpy().astype(str), 'консолидированный бюджет субъекта') >=
                                          0).any(0)][-1]]
        inc['revenue_id'] = inc[inc.columns[(find(
            inc.to_numpy().astype(str), '10000000000000000') >= 0).any(0)][0]]

        # replacing the old dataframe with a new 4-column dataframe:
        budget_dict[year][index][0] = inc[[
            'revenue_type_rus', 'revenue_id', 'revenue', 'filename'
        ]].dropna().reset_index(drop=True)

        # SPENDING

        # filling the total spending row in order to not lose it when executing dropna():
        spnd[spnd['Unnamed: 1'] == '200'] = spnd[spnd['Unnamed: 1'] ==
                                                 '200'].fillna(0)

        # locating the columns by the unique substrings...
        x = spnd.columns[(find(spnd.to_numpy().astype(str), 'в том числе') >=
                          0).any(0)][0]
        spnd['spending_type_rus'] = spnd[x]

        # ...or by their fixed position towards the located column:
        spnd['spending_id_1'] = spnd.iloc[:, spnd.columns.get_loc(x) + 3]

        if year < 2015:
            spnd['spending_id_2'] = spnd.iloc[:, spnd.columns.get_loc(x) + 6]
        else:
            spnd['spending_id_2'] = spnd.iloc[:, spnd.columns.get_loc(x) + 5]

        spnd['spending'] = spnd[spnd.columns[(find(spnd.to_numpy().astype(str),
                                                   'ванный бюджет субъекта') >=
                                              0).any(0)][-1]]

        # replacing the old dataframe with the new one:
        budget_dict[year][index][1] = spnd[[
            'spending_type_rus', 'spending_id_1', 'spending_id_2', 'spending',
            'filename'
        ]].dropna().reset_index(drop=True)

stop = perf_counter()
print('minutes passed:', (stop - start) / 60)

minutes passed: 9.29034633


All the revenue and spending dataframes in our dictionary are now readable (in Russian) and have an identical structure:

In [5]:
print(budget_dict[2021][33][0].head(3))
print()
print(budget_dict[2021][60][1].head(3))

                 revenue_type_rus         revenue_id              revenue  \
0  налоговые и неналоговые доходы  10000000000000000  225286897616.040009   
1       налоги на прибыль, доходы  10100000000000000  159607593524.170013   
2    налог на прибыль организаций  10101000000000110   90530315210.080002   

                                            filename  
0  ./budget_data/reg\reg_2021\34_01.01.2022_05033...  
1  ./budget_data/reg\reg_2021\34_01.01.2022_05033...  
2  ./budget_data/reg\reg_2021\34_01.01.2022_05033...  

                                   spending_type_rus  spending_id_1  \
0                           расходы бюджета - всего             0.0   
1                        общегосударственные вопросы          100.0   
2  функционирование высшего должностного лица суб...          102.0   

   spending_id_2            spending  \
0            0.0  185744536755.98999   
1            0.0  11679159478.379999   
2            0.0        105655168.61   

                        

### [Concatinating 1,870 dataframes](#contents) <a class="anchor" id="making_two_dataframes"></a>

Now we'll create lists out of our dataframes and then concatenate them within a nested loop:

In [6]:
year_list_rev = []  # a list of future 2011, 2012, etc. dataframes
year_list_spnd = []

for year in years:

    df_list_rev = []  # a list of 85 regional dataframes for the year
    df_list_spnd = []

    for index in range(len(budget_dict[year])):

        x = budget_dict[year][index][0]
        y = budget_dict[year][index][1]

        df_list_rev.append(
            x)  # appending each dataframe for the specific year to a list
        df_list_spnd.append(y)

    df_year_rev = pd.concat(
        df_list_rev, ignore_index=True
    )  # concatinating all the data frames for the specific year
    df_year_spnd = pd.concat(df_list_spnd, ignore_index=True)

    df_year_rev[
        'year'] = year  # appending a column with a year number to each year's dataframe
    df_year_spnd['year'] = year

    year_list_rev.append(
        df_year_rev)  # appending each unified data frame to a list
    year_list_spnd.append(df_year_spnd)

revenue = pd.concat(
    year_list_rev, ignore_index=True
)  # concatinating all revenue data frames into a single one
spending = pd.concat(year_list_spnd, ignore_index=True)  # same for spending

Now we can explore our unified data frames. Each has a "year" and a "filename" column that serve as indices for us, identifying a particular year and region, respectively.

In [7]:
print(revenue.info())
print()
print(spending.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566250 entries, 0 to 566249
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   revenue_type_rus  566250 non-null  object
 1   revenue_id        566250 non-null  object
 2   revenue           566250 non-null  object
 3   filename          566250 non-null  object
 4   year              566250 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 21.6+ MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320905 entries, 0 to 1320904
Data columns (total 6 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   spending_type_rus  1320905 non-null  object 
 1   spending_id_1      1320905 non-null  float64
 2   spending_id_2      1320905 non-null  float64
 3   spending           1320905 non-null  object 
 4   filename           1320905 non-null  object 
 5   year               1320905 non-null  

### [Tidying the data](#contents) <a class="anchor" id="tidying_the_data"></a>

Here we'll extract the region numbers from the filename column, and put the data types in order:

In [8]:
# extracting the 2-digit region number into a new column and getting rid of the filename column:
revenue['region_id'] = [int(x[27:29]) for x in revenue['filename']]
revenue = revenue.drop('filename', axis=1)

spending['region_id'] = [int(x[27:29]) for x in spending['filename']]
spending = spending.drop('filename', axis=1)

# cleaning the revenue and spending columns and making them numeric:
revenue['revenue'] = revenue['revenue'].replace(
    'консолидированный бюджет субъекта рф и бюджета территориального государственного внебюджетного фонда',
    np.nan,
    regex=True)
revenue['revenue'] = revenue['revenue'].replace(',', '.', regex=True)
revenue['revenue'] = revenue['revenue'].replace('\xa0', '', regex=True)
revenue['revenue'] = pd.to_numeric(revenue['revenue'])

spending['spending'] = spending['spending'].replace(' ', '', regex=True)
spending['spending'] = spending['spending'].replace(',', '.', regex=True)
spending['spending'] = spending['spending'].replace('\xa0', '', regex=True)
spending['spending'] = pd.to_numeric(spending['spending'])

# fixing the id columns types:
spending[['spending_id_1',
          'spending_id_2']] = spending[['spending_id_1',
                                        'spending_id_2']].astype('int')

# replacing some noise from the revenue id column:
revenue['revenue_id'] = revenue['revenue_id'].replace(' ', '', regex=True)
revenue = revenue[revenue['revenue_id'] != "х"]

We also need the key budget items translated into English. Here I'll use the data collected from the Minfin documents, which contain translations for the main revenue and spending items.

In [9]:
codes_revenue = pd.read_excel('additional_data/budget_codes_eng.xlsx')
codes_spending = pd.read_excel('additional_data/budget_codes_eng.xlsx',
                               sheet_name=1)

codes_revenue['revenue_indicator'] = codes_revenue[
    'revenue_indicator'].str.lower()
codes_spending['spending_indicator'] = codes_spending[
    'spending_indicator'].str.lower()

codes_revenue['code'] = codes_revenue['code'].apply(
    "{:.0f}".format).astype('str')

revenue = revenue.set_index('revenue_id').join(
    codes_revenue.rename(columns={
        'code': 'revenue_id'
    }).set_index('revenue_id'),
    how='left').reset_index().rename(
        columns={'revenue_indicator': 'revenue_type_eng'})

spending = spending.set_index('spending_id_1').join(
    codes_spending.rename(columns={
        'code': 'spending_id_1'
    }).set_index('spending_id_1'),
    how='left').reset_index().rename(
        columns={'spending_indicator': 'spending_type_eng'})

This is how our dataframes look like at the moment:

In [10]:
print(revenue.head(3))
print()
print(spending.tail(3))

          revenue_id                revenue_type_rus       revenue  year  \
0  10000000000000000  налоговые и неналоговые доходы  1.071983e+11  2011   
1  10000000000000000  налоговые и неналоговые доходы  2.206760e+10  2011   
2  10000000000000000  налоговые и неналоговые доходы  1.859744e+10  2011   

   region_id          revenue_type_eng  
0          1  tax and non-tax revenues  
1          2  tax and non-tax revenues  
2          3  tax and non-tax revenues  

         spending_id_1                                  spending_type_rus  \
1320902           1403                                           субсидии   
1320903           1403  субсидии, за исключением субсидий на софинанси...   
1320904           1403                       иные межбюджетные трансферты   

         spending_id_2      spending  year  region_id  \
1320902            520  3.500000e+08  2021         95   
1320903            521  3.500000e+08  2021         95   
1320904            540  1.234171e+09  2021        

## [Datasets 2-5: Population, Earnings, Poverty, USDRUB](#contents) <a class="anchor" id="datasets_2_5"></a>

To compare the changes in regional budget spending, we need to know how much money is spent per capita. It'd also be useful to take the USDRUB yearly exchange rate into account, as we'll analyse the period when it grew sharply. I also add the data on real income growth, as well as average income per capita in 2011–2021, to compare the possible changes in the regions' dwellers' welfare.

Moreover, we need to transform regions' codes in dataframes into their actual Russian and English names.

In [11]:
# Data on the population and income from the Federal Statistics Service + unique budget codes extracted from the Federal
# Treasury files and appended to it:
regions = pd.read_excel('additional_data/ru_regions_names_and_pop.xlsx',
                        index_col=0)
real_income = pd.read_excel('additional_data/real_income_regions.xlsx',
                            index_col=0)
income_percap = pd.read_excel('additional_data/income_per_cap_regions.xlsx',
                              index_col=0)
poverty = pd.read_excel('additional_data/poverty_regions.xlsx', index_col=0)

# Data on yearly USDRUB eachange rate, from the Central Bank of Russia:
rubusd = [[2011, 29.3925], [2012, 31.088], [2013, 31.8542], [2014, 38.4375],
          [2015, 60.9579], [2016, 67.0349], [2017, 58.3529], [2018, 62.7091],
          [2019, 64.7362], [2020, 72.1464], [2021, 73.6541]]
rubusd = pd.DataFrame(rubusd, columns=['year', 'rub_usd']).set_index('year')

In [12]:
print(regions.head(2))
print()
print(real_income.head(2))
print()
print(income_percap.head(2))
print()
print(poverty.head(2))
print()
print(rubusd.head(2))

                region_eng   capital      type    region_rus       2011  \
budget_code                                                               
1            Bashkortostan       Ufa  Republic  Башкортостан  4068165.0   
2                 Buryatia  Ulan-Ude  Republic       Бурятия   971464.0   

                  2012       2013     2014     2015     2016     2017  \
budget_code                                                             
1            4062601.0  4065327.0  4070841  4071525  4069018  4065132   
2             971600.0   972835.0   976177   980389   983209   984322   

                2018     2019     2020     2021  
budget_code                                      
1            4057148  4044578  4025968  4007732  
2             983891   984604   985683   984029  

            region_eng region_rus   2011   2012   2013   2014  2015   2016  \
budget_code                                                                  
95              Russia     Россия  101.2  105.8  

### [Transforming and appending the data](#contents) <a class="anchor" id="transforming_the_data"></a>

There's a separate column for each year in these dataframes, so we need to transform them into long format before joining them to the budget data.

In [13]:
# fixing the data types
regions = regions.fillna(0)
regions[[2011, 2012, 2013]] = regions[[2011, 2012, 2013]].astype('int')

income_percap[['2011',
               '2012']] = income_percap[['2011',
                                         '2012']].fillna(0).astype('int')

poverty = poverty.rename(
    columns={
        '2011': 2011,
        '2012': 2012,
        '2013': 2013,
        '2014': 2014,
        '2015': 2015,
        '2016': 2016,
        '2017': 2017,
        '2018': 2018,
        '2019': 2019,
        '2020': 2020,
        '2021': 2021
    })

# transforming the population dataframe
reg_names = pd.melt(regions.reset_index(),
                    id_vars=['budget_code', 'region_eng', 'region_rus'],
                    value_vars=list(range(2011, 2022)))
reg_names = reg_names.rename(columns={
    'variable': 'year',
    'value': 'population'
}).set_index(['budget_code', 'year'])
reg_names = reg_names.applymap(lambda s: s.lower() if type(s) == str else s)

# transforming the poverty dataframe
reg_poverty = pd.melt(poverty.reset_index(),
                      id_vars=['budget_code', 'region_eng', 'region_rus'],
                      value_vars=list(range(2011, 2022)))
reg_poverty = reg_poverty.rename(columns={
    'variable': 'year',
    'value': 'poverty'
}).set_index(['budget_code', 'year'])
reg_poverty = reg_poverty.applymap(lambda s: s.lower()
                                   if type(s) == str else s)

# transforming the real income dataframe
reg_realinc = pd.melt(real_income.reset_index(),
                      id_vars=['budget_code', 'region_eng', 'region_rus'],
                      value_vars=[str(x) for x in range(2011, 2022)])
reg_realinc = reg_realinc.rename(columns={
    'variable': 'year',
    'value': 'real_income'
})
reg_realinc['year'] = pd.to_numeric(reg_realinc['year']).astype('int')
reg_realinc = reg_realinc.set_index(['budget_code', 'year'])
reg_realinc = reg_realinc.applymap(lambda s: s.lower()
                                   if type(s) == str else s)

# transforming the income per capita dataframe
reg_incpercap = pd.melt(income_percap.reset_index(),
                        id_vars=['budget_code', 'region_eng', 'region_rus'],
                        value_vars=[str(x) for x in range(2011, 2022)])
reg_incpercap = reg_incpercap.rename(columns={
    'variable': 'year',
    'value': 'income_per_cap'
})
reg_incpercap['year'] = pd.to_numeric(reg_incpercap['year']).astype('int')
reg_incpercap = reg_incpercap.set_index(['budget_code', 'year'])
reg_incpercap = reg_incpercap.applymap(lambda s: s.lower()
                                       if type(s) == str else s)

# joining new data to the regional dataframes
revenue = revenue.rename(columns={
    'region_id': 'budget_code'
}).set_index(['budget_code', 'year']).join([
    reg_names, reg_realinc[['real_income']], reg_incpercap[['income_per_cap']],
    reg_poverty[['poverty']]
],
                                           how='left')
spending = spending.rename(columns={
    'region_id': 'budget_code'
}).set_index(['budget_code', 'year']).join([
    reg_names, reg_realinc[['real_income']], reg_incpercap[['income_per_cap']],
    reg_poverty[['poverty']]
],
                                           how='left')
revenue = revenue.reset_index().drop('budget_code', axis=1)
spending = spending.reset_index().drop('budget_code', axis=1)
revenue = revenue.set_index('year').join(rubusd).reset_index()
spending = spending.set_index('year').join(rubusd).reset_index()

revenue['population'] = revenue['population'].astype('int')
spending['population'] = spending['population'].astype('int')

In [14]:
print(revenue.head(3))
print()
print(spending.head(3))

   year         revenue_id                revenue_type_rus       revenue  \
0  2011  10000000000000000  налоговые и неналоговые доходы  1.071983e+11   
1  2011  10100000000000000       налоги на прибыль, доходы  6.398166e+10   
2  2011  10101000000000110    налог на прибыль организаций  3.041435e+10   

           revenue_type_eng     region_eng    region_rus  population  \
0  tax and non-tax revenues  bashkortostan  башкортостан     4068165   
1    profit taxes, revenues  bashkortostan  башкортостан     4068165   
2                       NaN  bashkortostan  башкортостан     4068165   

   real_income  income_per_cap  poverty  rub_usd  
0         99.7           19029     12.6  29.3925  
1         99.7           19029     12.6  29.3925  
2         99.7           19029     12.6  29.3925  

   year  spending_id_1                                  spending_type_rus  \
0  2011              0                           расходы бюджета - итого    
1  2011            100                        о

## [Dataset 6: Taxes](#contents) <a class="anchor" id="dataset_6"></a>

And here we've come to the most tricky part. There are several regions in Russia that collect huge amounts of taxes that don't eventually go to their own budgets but to the federal budget directly. So we don't see this money in our regional budget data, only the whole sum in the federal budget, not divided by regions. To know how much money the regions collect but don't get, we need to combine the budget data with the tax data from the Ministry of Finance.

This is how the dataframe from the Ministry looks like:

In [15]:
taxes = pd.read_excel('additional_data/taxes_to_budgets.xls')
taxes = taxes.rename(columns={'Unnamed: 0': 'region', 'Unnamed: 1': 'tax'})
taxes.head(3)

,region,tax,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,001. РОССИЙСКАЯ ФЕДЕРАЦИЯ,"Акцизы по подакцизным товарам (продукции), вво...",9978.0,7963.0,19591.0,46894.0,190566.0,425880.0,1950287.0,6453644.0,11424419.0,6.511568e+06,3.294184e+06,2.258882e+06,1.187403e+06,1.434696e+06,4.076034e+06,1730038.0
1,001. РОССИЙСКАЯ ФЕДЕРАЦИЯ,Акцизы по подакцизным товарам производимым на ...,253319846.0,289880222.0,314683718.0,327409405.0,441364830.0,603879883.0,783641696.0,952474791.0,998952392.0,1.014397e+09,1.293879e+09,1.521270e+09,1.493163e+09,1.277475e+09,1.833163e+09,675256532.0
2,001. РОССИЙСКАЯ ФЕДЕРАЦИЯ,"Акцизы по подакцизным товарам, производимым на...",160085812.0,181036272.0,189447895.0,245698130.0,327466372.0,372100000.0,441776441.0,491428796.0,478122999.0,4.865078e+08,6.617230e+08,6.116993e+08,6.324415e+08,7.552984e+08,7.979090e+08,950137998.0


#### Here we have:

- the number (not matching the Treasury budget codes) and the name of the region;

- the name of the tax, containing the clarification of the budget it was transferred to (the federal budget, the regional budget, the federal state non-budget fund, and the regional state non-budget fund);

- the transferred sum for each year.

First of all, let's extract the type of budget substring to a separate column:

In [16]:
# converting all strings to lower case
taxes = taxes.applymap(lambda s: s.lower() if type(s) == str else s)

# assigning variables to filter rows by substrings:
total = taxes['tax'].str.contains(
    "- всего", case=False)  # "всего" = "total sum transferred"
federal = taxes['tax'].str.contains("в федеральный бюджет",
                                    case=False)  # = "to federal budget"
regional = taxes['tax'].str.contains("бюджеты субъектов",
                                     case=False)  # = "to regional budgets"
federal_funds = taxes['tax'].str.contains(
    "фонда", case=False)  # = "to federal non-budget funds"
regional_funds = taxes['tax'].str.contains(
    "территориальных фондов", case=False)  # = "to regional non-budget funds"

# creating a new "budget" column with these variables:
taxes.loc[total, 'budget'] = 'total'
taxes.loc[federal, 'budget'] = 'federal'
taxes.loc[regional, 'budget'] = 'regional'
taxes.loc[federal_funds, 'budget'] = 'federal_funds'
taxes.loc[regional_funds, 'budget'] = 'regional_funds'

### [The assignment of codes to taxes](#contents) <a class="anchor" id="codes_to_taxes"></a>

To join tax data to the budget dataframe, a unique id should be assigned for each tax. I took these ids from the Russian Minfin documents. Not all of them are present in the budget code file that has been used earlier in this notebook.

In [17]:
# assigning variables to filter rows by taxes names:

for substring, tax_code, tax_eng, tax_rus in zip([
        "акцизы по подакцизным товарам \(\продукции\)\, ввозимым на территорию российской федерации",
        "производимым на территории российской федерации", "водный налог",
        "государственная пошлина", "единый налог на вмененный доход",
        "единый сельскохозяйственный налог",
        "налог на добавленную стоимость на товары \(\работы, услуги\)\, реализуемые",
        "налог на добавленную стоимость на товары, ввозимые",
        "налог на добычу газа", "налог на добычу газового конденсата",
        "налог на добычу нефти", "налог на добычу полезных ископаемых",
        "налог на дополнительный доход от добычи углеводородного сырья",
        "налог на доходы физических лиц", "налог на игорный бизнес",
        "налог на имущество",
        "налог на прибыль организаций при выполнении соглашений о разделе продукции",
        "налог на профессиональный доход",
        "налог, взимаемый в виде стоимости патента",
        "налог, взимаемый в связи с применением патентной",
        "налог, взимаемый в связи с применением упрощенной системы",
        "платежи за пользование недрами", "роялти", "животного мира",
        "транспортный налог", "поступило"
], [
        '10402000010000110', '10302000010000110', '10703000010000110',
        '10800000000000000', '10502000020000110', '10503000010000110',
        '10301000010000110', '10401000010000110', '10701012010000110',
        '10701013010000110', '10701011010000110', '10701000010000110',
        '10705000010000110', '10102000010000110', '10605000020000110',
        '10600000000000000', '10101020010000110', '10506000010000110',
        '10911000020000110', '10504000020000110', '10501000000000110',
        '10903060010000110', '10702000010000110', '10704000010000110',
        '10604000020000110', '00000000000000000'
], [
        'excises on imported goods', 'excises', 'water tax', 'state duty',
        'presumptive tax', 'unified agricultural tax', 'vat on sales',
        'vat on import', 'gas extraction tax', 'gas condensate extraction tax',
        'oil extraction tax', 'minerals extraction tax',
        'additional income from hydrocarbon extraction tax',
        'personal income tax', 'gambling tax', 'property taxes',
        'corporate income tax on the implementation of oil and gas fields development agreements',
        'professional income tax',
        'patent cost via the simplified taxation system',
        'patent taxation system', 'simplified tax system',
        'subsoil use payments', 'regular mining payments',
        'biological resources use fee', 'transport tax', 'total tax revenue'
], [
        'акцизы по ввозимым товарам', 'акцизы по производимым товарам',
        'водный налог', 'государственная пошлина',
        'единый налог на вмененный доход', 'единый сельскохозяйственный налог',
        'ндс на реализуемые товары', 'ндс на ввозимые товары',
        'налог на добычу газа', 'налог на добычу газового конденсата',
        'налог на добычу нефти', 'налог на добычу полезных ископаемых',
        'налог на дополнительный доход от добычи углеводородного сырья',
        'ндфл', 'налог на игорный бизнес', 'налог на имущество',
        'налог на прибыль организаций при выполнении соглашений о разработке месторождений нефти и газа',
        'налог на профессиональный доход',
        'налог в виде стоимости патента в связи с применением упрощенной системы налогообложения',
        'налог взимаемый в связи с применением патентной системы налогообложения',
        'налог на профессиональный доход в связи с применением упрощенной системы налогообложения',
        'платежи за пользование недрами',
        'регулярные платежи за добычу полезных ископаемых (роялти)',
        'сборы за пользование объктами животного мира', 'транспортный налог',
        'всего поступило налогов'
]):
    taxes.loc[taxes['tax'].str.contains(substring, case=False),
              ['tax_code', 'tax_eng', 'tax_rus']] = [
                  tax_code, tax_eng, tax_rus
              ]

taxes.loc[
    taxes['tax'].str.contains("налог на прибыль организаций", case=False) &
    (~taxes['tax'].str.contains("при выполнении соглашений", case=False)),
    ['tax_code', 'tax_eng', 'tax_rus']] = [
        '10101000000000110', 'corporate income tax full',
        'налог на прибыль организаций'
    ]

Now that we've added all the necessary columns, let's transform the dataframe:

In [18]:
taxes = taxes.drop('tax', axis=1)

# first, we create a long data from wide:
taxes_1 = pd.melt(
    taxes,
    id_vars=['region', 'tax_rus', 'tax_eng', 'tax_code', 'budget'],
    value_vars=[str(x) for x in range(2011, 2022)])
taxes_1 = taxes_1.rename(columns={'variable': 'year', 'value': 'sum'})
taxes_1['sum'] = taxes_1[
    'sum'] * 1000  # Minfin gives sums in RUB'000, the Treasury in RUB

# next, we create wide data, breaking the 'budget' column into separate columns for each budget type:
taxes_fin = taxes_1.pivot_table(
    index=['year', 'region', 'tax_code', 'tax_rus', 'tax_eng'],
    columns='budget',
    values='sum').fillna(0).reset_index()

print('Final table')
print()
print(taxes_fin.head(3))

Final table

budget  year                     region           tax_code  \
0       2011  001. российская федерация  00000000000000000   
1       2011  001. российская федерация  10101000000000110   
2       2011  001. российская федерация  10101020010000110   

budget                                            tax_rus  \
0                                 всего поступило налогов   
1                            налог на прибыль организаций   
2       налог на прибыль организаций при выполнении со...   

budget                                            tax_eng       federal  \
0                                       total tax revenue  4.480464e+12   
1                               corporate income tax full  3.426018e+11   
2       corporate income tax on the implementation of ...  4.095270e+08   

budget  federal_funds      regional  regional_funds         total  
0                 0.0  5.239135e+12             0.0  9.719599e+12  
1                 0.0  1.927714e+12             0.0  2.2

### [Extracting regions' names](#contents) <a class="anchor" id="extracting_names"></a>

We also need to get a region column in a unified format so that we can join the table to the main dataframe. Here I filter a messy region column by substrings to create a new region_eng column identical to the one from the budget dataframe:

In [19]:
for substring, eng_name in zip([
        "башкортостан", "бурятия", "дагестан", "кабардино", "калмыкия",
        "карелия", "коми", "марий", "мордовия", "осетия", "татарстан", "тыва",
        "удмуртская", "ингушетия", "чувашская", "якутия", "алтайский",
        "краснодарский", "красноярский", "приморский", "ставропольский",
        "хабаровский", "амурская", "архангельская", "астраханская",
        "белгородская", "брянская", "владимирская", "волгоградская",
        "вологодская", "воронежская", "нижегородская", "ивановская",
        "иркутская", "калининградская", "тверская", "калужская", "камчатский",
        "кемеровская", "кировская", "костромская", "самарская", "курганская",
        "курская", "ленинградская", "липецкая", "магаданская", "московская",
        "мурманская", "новгородская", "новосибирская", "оренбургская",
        "орловская", "пензенская", "пермский", "псковская", "ростовская",
        "рязанская", "саратовская", "сахалинская", "свердловская",
        "смоленская", "тамбовская", "томская", "тульская", "тюменская",
        "ульяновская", "челябинская", "ярославская", "петербург", "москва",
        "севастополь", "крым", "адыгея", "еврейская", "карачаево", "хакасия",
        "ханты", "чукотский", "ямало", "забайкальский", "чеченская",
        "российская федерация"
], [
        'Bashkortostan', 'Buryatia', 'Dagestan', 'Kabardino-Balkaria',
        'Kalmykia', 'Karelia', 'Komi', 'Mariy El', 'Mordovia',
        'North Osetia - Alania', 'Tatarstan', 'Tyva', 'Udmurtia', 'Ingushetia',
        'Chuvashia', 'Sakha (Yakutia)', 'Altai Krai', 'Krasnodarsky Krai',
        'Krasnoyarsky Krai', 'Primorsky Krai', 'Stavropolsky Krai',
        'Khabarovsky Krai', 'Amur Oblast', 'Arkhangelsk Oblast',
        'Astrakhan Oblast', 'Belgorod Oblast', 'Bryansk Oblast',
        'Vladimir Oblast', 'Volgograd Oblast', 'Vologda Oblast',
        'Voronezh Oblast', 'Nizhny Novgorod Oblast', 'Ivanovo Oblast',
        'Irkutsk Oblast', 'Kaliningrad Oblast', 'Tver Oblast', 'Kaluga Oblast',
        'Kamchatsky Krai', 'Kemerovo Oblast', 'Kirov Oblast',
        'Kostroma Oblast', 'Samara Oblast', 'Kurgan Oblast', 'Kursk Oblast',
        'Leningrad Oblast', 'Lipetsk Oblast', 'Magadan Oblast',
        'Moscow Oblast', 'Murmansk Oblast', 'Novgorod Oblast',
        'Novosibirsk Oblast', 'Orenburg Oblast', 'Orel Oblast', 'Penza Oblast',
        'Permsky Krai', 'Pskov Oblast', 'Rostov Oblast', 'Ryazan Oblast',
        'Saratov Oblast', 'Sakhalin Oblast', 'Sverdlov Oblast',
        'Smolensk Oblast', 'Tambov Oblast', 'Tomsk Oblast', 'Tula Oblast',
        'Tyumen Oblast', 'Ulyanovsk Oblast', 'Chelyabinsk Oblast',
        'Yaroslavl Oblast', 'Saint Petersburg', 'Moscow', 'Sevastopol',
        'Crimea', 'Adygea', 'Jewish Autonomous Oblast',
        'Karachaevo-Cherkessia', 'Hakasia',
        'Khanty-Mansiysk Autonomous Okrug – Ugra', 'Chukotka Autonomous Okrug',
        'Yamalo-Nenets Autonomous Okrug', 'Zabaykalsky Krai', 'Chechnya',
        'Russia'
]):
    taxes_fin.loc[taxes_fin['region'].str.contains(substring, case=False),
                  'region_eng'] = eng_name

taxes_fin.loc[(taxes_fin['region'].str.contains("омская", case=False) &
               (~taxes_fin['region'].str.contains("костро", case=False))),
              'region_eng'] = 'Omsk Oblast'
taxes_fin.loc[(taxes_fin['region'].str.contains("алтай", case=False) &
               (~taxes_fin['region'].str.contains("край", case=False))),
              'region_eng'] = 'Altai'
taxes_fin.loc[(taxes_fin['region'].str.contains("ненецкий", case=False) &
               (~taxes_fin['region'].str.contains("ямало", case=False))),
              'region_eng'] = 'Nenets Autonomous Okrug'

taxes_fin['region_eng'] = taxes_fin['region_eng'].str.lower()
taxes_fin = taxes_fin.drop('region', axis=1)

### [Joining taxes to budgets](#contents) <a class="anchor" id="joining_taxes"></a>

In [20]:
# The first column calculates the total sum of taxes transferred to the federal level, namely, to the federal budget and to
# the budgets of federal state non-budget funds

taxes_fin['tax_to_fed'] = taxes_fin['federal'] + taxes_fin['federal_funds']

# The second column estimates the sum of taxes transferred to the region solely (without regional non-budget fund). We need
# this column only to compare the budget and the tax data for disparities between the transferred and registered money.
taxes_fin = taxes_fin.rename(columns={'regional': 'tax_to_region'})
taxes_fin = taxes_fin.drop(
    ['federal', 'federal_funds', 'regional_funds', 'total'], axis=1)

# joining the dataframes
taxes_fin['year'] = taxes_fin['year'].astype('int')
taxes_fin = taxes_fin.rename(columns={'tax_code': 'revenue_id'})
taxes_fin = taxes_fin.set_index(['year', 'region_eng', 'revenue_id'])

revenue_fin = revenue.set_index(['year', 'region_eng', 'revenue_id'
                                 ]).join(taxes_fin, how='outer').reset_index()

# filling the missing revenue names in the main dataframe with those from the taxes data
revenue_fin['revenue_type_rus'] = revenue_fin['revenue_type_rus'].fillna(
    revenue_fin['tax_rus'])
revenue_fin['revenue_type_eng'] = revenue_fin['revenue_type_eng'].fillna(
    revenue_fin['tax_eng'])
revenue_fin = revenue_fin.drop(['tax_rus', 'tax_eng'], axis=1)

# getting rid of some NaN values in population and income that appeared after appending the tax data
data_for_fill = revenue_fin.groupby(by=['year', 'region_eng'])[[
    'population', 'real_income', 'income_per_cap', 'poverty', 'rub_usd'
]].min()
data_for_fill = data_for_fill.rename(
    columns={
        'population': 'population_fill',
        'real_income': 'real_income_fill',
        'income_per_cap': 'income_per_cap_fill',
        'poverty': 'poverty_fill',
        'rub_usd': 'rub_usd_fill'
    })

revenue_fin = revenue_fin.set_index(['year', 'region_eng'
                                     ]).join(data_for_fill).reset_index()
revenue_fin['population'] = revenue_fin['population'].fillna(
    revenue_fin['population_fill'])
revenue_fin['real_income'] = revenue_fin['real_income'].fillna(
    revenue_fin['real_income_fill'])
revenue_fin['income_per_cap'] = revenue_fin['income_per_cap'].fillna(
    revenue_fin['income_per_cap_fill'])
revenue_fin['poverty'] = revenue_fin['poverty'].fillna(
    revenue_fin['poverty_fill'])
revenue_fin['rub_usd'] = revenue_fin['rub_usd'].fillna(
    revenue_fin['rub_usd_fill'])
revenue_fin = revenue_fin.drop([
    'population_fill', 'real_income_fill', 'income_per_cap_fill',
    'poverty_fill', 'rub_usd_fill'
],
                               axis=1)

revenue_fin.head()

,year,region_eng,revenue_id,revenue_type_rus,revenue,revenue_type_eng,region_rus,population,real_income,income_per_cap,poverty,rub_usd,tax_to_region,tax_to_fed
0,2011,adygea,00000000000000000,всего поступило налогов,NaN,total tax revenue,NaN,441193.0,106.6,14271.0,14.6,29.3925,5.363718e+09,1.074986e+09
1,2011,adygea,10000000000000000,налоговые и неналоговые доходы,6.523791e+09,tax and non-tax revenues,адыгея,441193.0,106.6,14271.0,14.6,29.3925,NaN,NaN
2,2011,adygea,10100000000000000,"налоги на прибыль, доходы",3.622449e+09,"profit taxes, revenues",адыгея,441193.0,106.6,14271.0,14.6,29.3925,NaN,NaN
3,2011,adygea,10101000000000110,налог на прибыль организаций,1.171432e+09,corporate income tax full,адыгея,441193.0,106.6,14271.0,14.6,29.3925,1.171432e+09,4.420800e+07
4,2011,adygea,10101010000000110,"налог на прибыль организаций, зачисляемый в бю...",1.171426e+09,corporate income tax,адыгея,441193.0,106.6,14271.0,14.6,29.3925,NaN,NaN


## [Checking for mismatches in data](#contents) <a class="anchor" id="mismatches"></a>

Now we have two columns in a data frame that should stand for the same sum of money:

- Tax_to_region column stands for the sum of taxes transferred to the regional budget according to the Federal Tax Service and Minfin.

- Revenue column stands for the accounted regional tax revenue.

Let's check whether they always match.

### [Checking for "disappeared" taxes](#contents) <a class="anchor" id="disappeared"></a>

First of all, we have some taxes that were transferred to the consolidated regional budget according to Minfin but didn't appear there according to the Federal Treasury.

Pivoting the data lets us conclude that this is true for certain regions, in particular, for Nenets autonomous okrug:

In [21]:
# sorting out all the taxes that went to regional budgets, accorging to Minfin
check_table = revenue_fin[revenue_fin['tax_to_region'].notnull()].query(
    'region_eng != "russia"')[['year', 'region_eng', 'revenue_id', 'revenue_type_rus', 'revenue_type_eng', 'revenue', 
                               'tax_to_region']]

# the number of taxes that were transferred to regional budgets according to Minfin, but didn't appear there according to the
# Federal Treasury; by regions
check_piv1 = pd.pivot_table(data=check_table[check_table['revenue'].isna()].query(
    'tax_to_region != 0.0 & revenue_type_eng != "total tax revenue"'), index='region_eng', columns='year',
                            values='tax_to_region', aggfunc='count')

check_piv1 = check_piv1.fillna(0)
check_piv1 = check_piv1.astype('int')
print()
print('The number of taxes that have been transferred to regions but not accounted for by them')
print()

check_piv1.head(50)


The number of taxes that have been transferred to regions but not accounted for by them



year,2011,2012,2013,2014,2016,2017,2020,2021
region_eng,,,,,,,,
adygea,2,0,0,0,0,0,0,0
altai,2,0,0,0,0,0,0,0
altai krai,2,0,0,0,0,0,0,0
amur oblast,2,0,0,0,0,0,0,0
arkhangelsk oblast,2,0,0,0,0,0,0,0
astrakhan oblast,2,0,0,0,0,0,0,0
bashkortostan,2,0,0,0,0,0,0,0
belgorod oblast,2,0,0,0,0,0,0,0
bryansk oblast,2,0,0,0,0,0,0,0


And there's also such a problem with two types of taxes, namely, presumptive tax and unified agricultural tax:

In [22]:
# the number of taxes that were transferred to regional budgets according to Minfin, but didn't appear there according to the
# Federal Treasury; by tax type
check_piv2 = pd.pivot_table(data=check_table[check_table['revenue'].isna()].query(
    'tax_to_region != 0.0  & revenue_type_eng != "total tax revenue"'), index='revenue_type_eng', columns='year', 
                            values='region_eng', aggfunc='count')

check_piv2 = check_piv2.fillna(0)
check_piv2 = check_piv2.astype('int')
print()
print('The number of regions where the taxes have been transferred to but not accounted for')
print()

check_piv2.head(50)


The number of regions where the taxes have been transferred to but not accounted for



year,2011,2012,2013,2014,2016,2017,2020,2021
revenue_type_eng,,,,,,,,
biological resources use fee,1,1,1,1,0,0,0,0
corporate income tax full,1,1,1,1,0,0,0,0
corporate income tax on the implementation of oil and gas fields development agreements,1,1,1,1,1,1,0,0
excises,1,1,1,0,0,0,0,0
minerals extraction tax,1,1,1,1,0,0,0,0
presumptive tax,83,0,0,0,0,0,0,0
professional income tax,0,0,0,0,0,0,1,1
regular mining payments,1,1,1,1,0,0,0,0
simplified tax system,0,0,0,1,0,0,0,0


### [Checking for discrepancies in sums of taxes](#contents) <a class="anchor" id="discrepancies"></a>

We'll also check whether there are significant discrepancies in the sums of transferred and accounted taxes. Let's consider 10% difference as significant and sort out our dataframe:

In [23]:
# sorting out rows where revenues and tax_to_region have a 10% difference and more
check_table2 = revenue_fin[revenue_fin['revenue'].notnull()].assign(
    NE=abs(revenue_fin.revenue)*0.9 > abs(revenue_fin.tax_to_region), 
    NE1=abs(revenue_fin.revenue)*1.1 < abs(revenue_fin.tax_to_region)).query(
    '(NE == True | NE1 == True) & revenue != 0.0 & tax_to_region != 0.0')

check_piv3 = pd.pivot_table(check_table2, index='revenue_type_eng', columns='year', values='region_eng', aggfunc='count')

check_piv3 = check_piv3.fillna(0)
check_piv3 = check_piv3.astype('int')
print()
print('The number of regions with mismatched transferred and accounted tax summings')
print()
check_piv3.head(50)


The number of regions with mismatched transferred and accounted tax summings



year,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
revenue_type_eng,,,,,,,,,,,
biological resources use fee,1,2,2,2,2,2,2,2,2,2,2
corporate income tax full,4,4,5,5,6,6,6,6,5,6,6
corporate income tax on the implementation of oil and gas fields development agreements,2,7,7,4,4,9,8,4,2,1,2
excises,81,80,80,77,80,82,80,82,80,82,81
gambling tax,7,1,1,1,1,1,0,1,1,3,1
minerals extraction tax,2,2,2,2,2,2,2,2,2,2,2
patent based simplified tax system,0,0,0,0,0,0,0,0,1,1,0
patent cost via the simplified taxation system,0,0,0,0,0,0,0,0,5,9,9
personal income tax,2,3,3,3,2,2,2,2,2,2,2


#### We have significant disparities within several taxes, respectively:

- corporate income tax on the implementation of oil and gas field development agreements (which is part of corporate income tax);

- excises on Russian goods;

- gambling tax, which is a part of property tax;

- state duty;

- patent cost via simplified taxation system.

Let's look at the particular regions where the summings didn't match:

In [24]:
check_piv4 = pd.pivot_table(check_table2.query('revenue_type_eng not in ("state duty", "excises")'), 
                            index=['revenue_type_eng','region_eng'], columns='year', values='revenue_id', aggfunc='count')

check_piv4 = check_piv4.fillna(0)
check_piv4 = check_piv4.astype('int')
print()
print('The mismatched transferred and accounted tax summings by type of tax and region')
print()
check_piv4.head(50)


The mismatched transferred and accounted tax summings by type of tax and region



year                                                                                        2011  \
revenue_type_eng                                   region_eng                                      
biological resources use fee                       arkhangelsk oblast                          0   
                                                   nenets autonomous okrug                     0   
                                                   omsk oblast                                 1   
corporate income tax full                          arkhangelsk oblast                          1   
                                                   khanty-mansiysk autonomous okrug – ugra     1   
                                                   nenets autonomous okrug                     0   
                                                   omsk oblast                                 0   
                                                   tyumen oblast                               1   
                                                   yamalo-nenets autonomous okrug              1   
corporate income tax on the implementation of o... altai                                       0   
                                                   altai krai                                  0   
                                                   chelyabinsk oblast                          0   
                                                   crimea                                      0   
                                                   dagestan                                    0   
                                                   irkutsk oblast                              0   
                                                   ivanovo oblast                              0   
                                                   khabarovsky krai                            0   
                                                   khanty-mansiysk autonomous okrug – ugra     0   
                                                   krasnodarsky krai                           0   
                                                   krasnoyarsky krai                           0   
                                                   kursk oblast                                0   
                                                   lipetsk oblast                              0   
                                                   mordovia                                    0   
                                                   nenets autonomous okrug                     0   
                                                   nizhny novgorod oblast                      0   
                                                   north osetia - alania                       0   
                                                   novgorod oblast                             0   
                                                   novosibirsk oblast                          0   
                                                   tatarstan                                   0   
                                                   tula oblast                                 1   
                                                   ulyanovsk oblast                            1   
                                                   volgograd oblast                            0   
                                                   vologda oblast                              0   
                                                   voronezh oblast                             0   
                                                   yaroslavl oblast                            0   
                                                   zabaykalsky krai                            0   
gambling tax                                       altai                                       1   
                                                   chuvashia                                   0   


#### The causes of the disparities

- Minfin rounds the tax but the Treasury does not, while the <b>amount is small</b> (so rounding reduces it significantly). We can just use the Treasury's value in such cases.

- There's <b>a redistribution of the revenue between sub-regions</b>, like between the Tyumen region, Yamal-Nenets, and Khanty-Mansiysk autonomous okrugs; between the Arkhangelsk region and Nenets autonomous okrug; and between Moscow and the Moscow region. If we sum up the money transferred to them and the money that was actually counted as revenue, these sums will match.

- Excises on goods produced in Russia. The Treasury <b>does not include part of the excise</b> registered by its territorial authorities into the report, so it is impossible to determine the exact amount of excise sent to the federal budget within the region (here's the explanation from one of the key Russian rating agencies (RU): https://www.acra-ratings.ru/research/2302/).  

- State duty. Minfin <b>reports only one type of this tax</b> (state fee for cases considered in general jurisdiction courts) as going to regional budgets. But according to the Federal Treasury, the region's actual revenue from state duty is much higher. According to the the Federal Treasury, a significant part of the money comes from another type of state duty, state registration, which is supposed to go to the federal budget according to the law. How much is actually transferred to the federal budget is not clear.

The main outcome: <b>certain items'll remain hidden from us, but the amounts of these "lost" money are not that significant.</b>

 ## [Dataset 7: Federal Budget](#contents) <a class="anchor" id="dataset_7"></a>

We'll also create two separate dataframes for federal revenues and spending. The process is mostly similar to transforming the regional budget data.

In [25]:
# creating a dictionary with two tables for each year between 2011 and 2021:

path_dir = r"./budget_data/fed"

files = glob.glob(os.path.join(path_dir, "*.xlsx"))

budget_dict = dict()

budget_dict = []

for f in files:
    year = int(f[18:22]) 
        
    x= pd.read_excel(f, sheet_name=1)
    x['filename'] = f
    y= pd.read_excel(f, sheet_name=3)
    y['filename'] = f
            
    budget_dict.append([x, y])

In [26]:
# extracting the columns:

for index in [0,1,2,5,6,7,8,9]:
        
    inc = budget_dict[index][0].applymap(lambda s: s.lower() if type(s) == str else s) 
    spnd = budget_dict[index][1].applymap(lambda s: s.lower() if type(s) == str else s)
        
    inc = inc.iloc[:, [0,2,3,4]]
    inc.columns = ['revenue_type_rus', 'revenue_id', 'revenue', 'year']
    inc['year'] = [x[18:22] for x in inc['year']] 
    
    budget_dict[index][0] = inc.dropna().reset_index(drop=True).drop(inc.index[0:4], axis=0).reset_index(drop=True)
        
    spnd = spnd.iloc[:, [0,2,3,4,5,8,10]]
    spnd.columns = ['spending_type_rus', 'spnd_id_1', 'spnd_id_2', 'spnd_id_3', 'spnd_id_4', 'spending', 'year']
    spnd['year'] = [x[18:22] for x in spnd['year']] 
    
    if index == 0:
        budget_dict[index][1] = spnd.drop([0,1,2,3,4,5,7], axis=0).reset_index(drop=True)
    else:
        budget_dict[index][1] = spnd.drop([0,1,2,3,4,6], axis=0).reset_index(drop=True)
    
# the 2021 Treasury dataframe is different from others (it's cumulative, not generalized report), so we'll 
# handle it separately;
# I also use cumulative reports for for 2014-2015 dataframes that have missing rows for an unknown reason

for index in [3,4,10]:
    
    inc = budget_dict[index][0].applymap(lambda s: s.lower() if type(s) == str else s)
    spnd = budget_dict[index][1].applymap(lambda s: s.lower() if type(s) == str else s)
    
    inc = inc.iloc[:, [0,3,5,9]]
    inc.columns = ['revenue_type_rus', 'revenue_id', 'revenue', 'year']
    inc['year'] = [x[18:22] for x in inc['year']] 
    
    budget_dict[index][0] = inc.dropna().reset_index(drop=True)  
    
    spnd = spnd.iloc[:, [0,2,3,4,5,7,9]]
    spnd.columns = ['spending_type_rus', 'spnd_id_1', 'spnd_id_2', 'spnd_id_3', 'spnd_id_4', 'spending', 'year']
    spnd['year'] = [x[18:22] for x in spnd['year']]
    
    budget_dict[index][1] = spnd.drop([0,1,2,3,4,6], axis=0).reset_index(drop=True)

In [27]:
# concatinating the dataframes:

df_list_rev = []
df_list_spnd = []
    
for index in range(len(budget_dict)):
        
    x = budget_dict[index][0]
    y = budget_dict[index][1]
        
    df_list_rev.append(x)
    df_list_spnd.append(y)
    
fed_rev = pd.concat(df_list_rev, ignore_index=True)
fed_spnd = pd.concat(df_list_spnd, ignore_index=True)

In [28]:
# fixing the data types:

fed_rev['revenue'] = fed_rev['revenue'].replace('\xa0', '', regex=True)
fed_rev['revenue'] = fed_rev['revenue'].replace(' ', '', regex=True)
fed_rev['revenue'] = fed_rev['revenue'].replace(',', '.', regex=True)
fed_rev['revenue'] = pd.to_numeric(fed_rev['revenue'])

fed_spnd['spending'] = fed_spnd['spending'].replace(',', '.', regex=True)
fed_spnd['spending'] = fed_spnd['spending'].replace(' ', '', regex=True)
fed_spnd['spending'] = fed_spnd['spending'].replace('\xa0', '', regex=True)
fed_spnd['spending'] = pd.to_numeric(fed_spnd['spending'])

fed_rev['revenue_id'] = fed_rev['revenue_id'].replace(' ', '', regex=True)
fed_rev['revenue_id'] = fed_rev['revenue_id'].replace('x', '00000000000000000', regex=True)

fed_rev['year'] = pd.to_numeric(fed_rev['year'])
fed_spnd['year'] = pd.to_numeric(fed_spnd['year'])

# in some dataframes it's an English 'x' in some it's a Russian 'x':
fed_spnd['spnd_id_1'] = fed_spnd['spnd_id_1'].replace('х', '0', regex=True)
fed_spnd['spnd_id_1'] = fed_spnd['spnd_id_1'].replace('x', '0', regex=True)
fed_spnd['spnd_id_2'] = fed_spnd['spnd_id_2'].replace('x', '0', regex=True)
fed_spnd['spnd_id_2'] = fed_spnd['spnd_id_2'].replace('х', '0', regex=True)
fed_spnd['spnd_id_3'] = fed_spnd['spnd_id_3'].replace('х', '0', regex=True)
fed_spnd['spnd_id_4'] = fed_spnd['spnd_id_4'].replace('х', '0', regex=True)
fed_spnd['spnd_id_3'] = fed_spnd['spnd_id_3'].replace('x', '0', regex=True)
fed_spnd['spnd_id_4'] = fed_spnd['spnd_id_4'].replace('x', '0', regex=True)

fed_spnd['spnd_id_4'] = fed_spnd['spnd_id_4'].astype('str')
fed_spnd['spnd_id_4'] = fed_spnd['spnd_id_4'].str.split('.').str[0]
fed_spnd['spnd_id_4'] = fed_spnd['spnd_id_4'].replace('nan', np.nan, regex=True)

fed_spnd[['spnd_id_1', 'spnd_id_2', 'spnd_id_3', 'spnd_id_4']] = fed_spnd[['spnd_id_1', 'spnd_id_2', 'spnd_id_3', 'spnd_id_4']].fillna(0)

fed_spnd['spnd_id_1'] = pd.to_numeric(fed_spnd['spnd_id_1']).astype('Int64').astype('int')
fed_spnd['spnd_id_2'] = pd.to_numeric(fed_spnd['spnd_id_2']).astype('Int64').astype('int')
fed_spnd['spnd_id_4'] = pd.to_numeric(fed_spnd['spnd_id_4']).astype('Int64').astype('int')

In [29]:
# appending English names of budget items:
codes_revenue_1 = revenue_fin.query(
    'revenue_id != "00000000000000000"')[['revenue_id', 'revenue_type_eng']].groupby(
    by=['revenue_id', 'revenue_type_eng']).min().reset_index()
codes_revenue = codes_revenue.rename(columns={'code':'revenue_id', 'revenue_indicator':'revenue_type_eng'})
codes_revenue = pd.concat([codes_revenue_1, codes_revenue], ignore_index=True).drop_duplicates().sort_values(by='revenue_id')

codes_spending = pd.read_excel('additional_data/budget_codes_eng.xlsx', sheet_name=1)
codes_spending['spending_indicator'] = codes_spending['spending_indicator'].str.lower()

fed_rev = fed_rev.set_index('revenue_id').join(codes_revenue.set_index('revenue_id'), how='left').reset_index()
fed_spnd = fed_spnd.set_index('spnd_id_2').join(
    codes_spending.rename(columns={'code':'spnd_id_2'}).set_index('spnd_id_2'), how='left').reset_index().rename(
    columns={'spending_indicator':'spending_type_eng'})

In [30]:
# getting rid of noise:

fed_spnd = fed_spnd[~fed_spnd['spending_type_rus'].isnull()]
fed_spnd = fed_spnd[~fed_spnd['spending_type_rus'].str.contains('дефицит/профицит')]
fed_spnd = fed_spnd[~fed_spnd['spending_type_rus'].str.contains('результат исполнения федерального бюджета')]

## [Codification](#contents) <a class="anchor" id="codification"></a>

In this chunk, we'll <b>extract subcodes from the unique transaction codes</b> to ease future analysis.

#### Revenue Codes

According to the law, the unique revenue type code consists of several parts:

- digit 1: revenue group (own revenues/transfers from federal center)

- digits 2-3: revenue subgroup, e.g., income taxes, obligatory insurance premiums, sales, fines, etc.

- digits 4-5: revenue item, e.g. corporate income tax/personal income tax inside the income tax, or VAT/excises inside the goods and services tax.

- digits 6-8: revenue subitem, e.g., oil extraction tax inside minerals extraction tax inside natural resources use payments.

The link: http://www.consultant.ru/document/cons_doc_LAW_300629/a9ab78893f22265f5b6e21e3804d145a61399501/

The remaining digits define more particular features of a transaction and are not important for our analysis.

#### Spending Codes

Two parts of the spending type code have been extracted at the beginning of the notebook. The 1st part (spending_type_1) consists of 4 digits:

- digits 1-2: spending section (national defence, healthcare, etc.)

- digits 3-4: spending subsection (e.g., in-patient medical care/emergency care inside healthcare)

I extract these digits in the next chunk. We'll use them to aggregate data at different levels.

The 2nd part (spending_type_2) represents the type of spending (state personnel payments, capital investments, etc.). I don't use it in this particular project, but it'll stay in the dataframe.

In [31]:
# REGIONAL BUDGETS:

# extracting subcodes and making them numeric
revenue_fin['r1'] = [int(x[:1]) for x in revenue_fin['revenue_id']]
revenue_fin['r2'] = [int(x[1:3]) for x in revenue_fin['revenue_id']]
revenue_fin['r3'] = [int(x[3:5]) for x in revenue_fin['revenue_id']]
revenue_fin['r4'] = [int(x[5:8]) for x in revenue_fin['revenue_id']]

# transforming the spending code to extract substrings
spending['spending_id_1'] = spending['spending_id_1'].astype('str').str.zfill(4)

# extracting subcodes and making them numeric
spending['s1'] = [int(x[:2]) for x in spending['spending_id_1']]
spending['s2'] = [int(x[2:4]) for x in spending['spending_id_1']]

spending['spending_id_1'] = pd.to_numeric(spending['spending_id_1']).astype('int')

# FEDERAL BUDGET:

fed_rev['r1'] = [int(x[:1]) for x in fed_rev['revenue_id']]
fed_rev['r2'] = [int(x[1:3]) for x in fed_rev['revenue_id']]
fed_rev['r3'] = [int(x[3:5]) for x in fed_rev['revenue_id']]
fed_rev['r4'] = [int(x[5:8]) for x in fed_rev['revenue_id']]

fed_spnd['spnd_id_2'] = fed_spnd['spnd_id_2'].astype('str').str.zfill(4)

fed_spnd['s1'] = [int(x[:2]) for x in fed_spnd['spnd_id_2']]
fed_spnd['s2'] = [int(x[2:4]) for x in fed_spnd['spnd_id_2']]

fed_spnd['spnd_id_2'] = pd.to_numeric(fed_spnd['spnd_id_2']).astype('int')

In [32]:
revenue_fin.head()

,year,region_eng,revenue_id,revenue_type_rus,revenue,revenue_type_eng,region_rus,population,real_income,income_per_cap,poverty,rub_usd,tax_to_region,tax_to_fed,r1,r2,r3,r4
0,2011,adygea,00000000000000000,всего поступило налогов,NaN,total tax revenue,NaN,441193.0,106.6,14271.0,14.6,29.3925,5.363718e+09,1.074986e+09,0,0,0,0
1,2011,adygea,10000000000000000,налоговые и неналоговые доходы,6.523791e+09,tax and non-tax revenues,адыгея,441193.0,106.6,14271.0,14.6,29.3925,NaN,NaN,1,0,0,0
2,2011,adygea,10100000000000000,"налоги на прибыль, доходы",3.622449e+09,"profit taxes, revenues",адыгея,441193.0,106.6,14271.0,14.6,29.3925,NaN,NaN,1,1,0,0
3,2011,adygea,10101000000000110,налог на прибыль организаций,1.171432e+09,corporate income tax full,адыгея,441193.0,106.6,14271.0,14.6,29.3925,1.171432e+09,4.420800e+07,1,1,1,0
4,2011,adygea,10101010000000110,"налог на прибыль организаций, зачисляемый в бю...",1.171426e+09,corporate income tax,адыгея,441193.0,106.6,14271.0,14.6,29.3925,NaN,NaN,1,1,1,10


In [33]:
revenue_fin.to_csv('final_data/revenue.csv')
spending.to_csv('final_data/spending.csv')
fed_rev.to_csv('final_data/fed_rev.csv')
fed_spnd.to_csv('final_data/fed_spend.csv')

## [Creating a dataset for analysis](#contents) <a class="anchor" id="creating_the_dataset"></a>

Now we are ready to create dataframes for future analysis with only the (potentially) necessary indicators. It'll let us do historical analysis and track the Russian budget data if (when) new information is published by the authorities of the country.

In fact, we need two levels of data aggregation:

The country level, containing the key federal revenue and spending yearly indicators as well as the regional indicators aggregated by the whole country;

The regional level, containing the key revenue and spending yearly indicators for each region.

We'll create the coding system to filter the dataframe by different parameters:

#### GENERAL CODES:

- <b>i1</b>: the level of aggregation (1 = the region, 2 = the whole country);

- <b>i2</b>: the type of budget (1 = regional, 2 = federal);

- <b>i3</b>: the type of indicator (1 = revenue, 2 = spending, 6 = population, 7 = real income, 8 = income per capita, 9 = USDRUB).

#### REVENUE CODES:

- <b>r1</b>: revenue group (1 = own revenues, 2 = federal transfers to regions, 3 = regional taxes to the federal budget);

- <b>r2</b>: revenue type (1 = tax, 2 = nontax);

- <b>r3</b>: revenue subgroup;

- <b>r4</b>: revenue item;

- <b>r5</b>: revenue subitem.

#### SPENDING CODES:

- <b>s1</b>: spending section;

- <b>s2</b>: spending subsection.

In [34]:
fed_spnd['spnd_id_1'] = fed_spnd['spnd_id_1'].astype('int')

# COUNTRY DATA **************************************************************************************

# REGIONS/REVENUE

# Regional money flows: own revenues, taxes fo federation, transfers from federation
reg_revenue_1 = revenue_fin.query(
    '(r1 in (0,1) & r2 == 0 & region_eng == "russia") | (r1 == 2 & r2 == 2 & r3 == 0 & region_eng == "russia")'
)[['year', 'revenue_id', 'revenue',
   'tax_to_fed']].reset_index(drop=True).set_index('year')
reg_revenue_1['revenue'] = reg_revenue_1['revenue'].fillna(
    reg_revenue_1['tax_to_fed'])
reg_revenue_1 = reg_revenue_1.drop('tax_to_fed', axis=1)
reg_revenue_1 = reg_revenue_1.reset_index().pivot_table(index='revenue_id',
                                                        columns='year',
                                                        values='revenue')
# no cumulative data for all Russia in 2011, so we count sums for regions and add them:
reg_revenue_1.loc['10000000000000000', 2011] = revenue_fin.query(
    'year == 2011 & r1 == 1 & r2 == 0')['revenue'].sum()
reg_revenue_1.loc['20200000000000000', 2011] = revenue_fin.query(
    'year == 2011 & r1 == 2 & r2 == 2 & r3 == 0')['revenue'].sum()
reg_revenue_1 = reg_revenue_1.rename(
    index={
        '00000000000000000': 'tax_to_fed',
        '10000000000000000': 'reg_own_revenue',
        '20200000000000000': 'transfers_to_reg'
    })
reg_revenue_1['i2'] = 1  # 1 for regions 2 for federal budget
reg_revenue_1['i3'] = 1  # 1 for revenues 2 for spending
reg_revenue_1['r1'] = [
    3, 1, 2
]  # 1 for own revenues, 2 for transfers, 3 for federal tax
reg_revenue_1['r2'] = 0  # 1 for tax 2 for nontax own revenue
reg_revenue_1['r3'] = 0  # revenue subgroup
reg_revenue_1['r4'] = 0  # revenue item
reg_revenue_1['r5'] = 0  # revenue subitem
reg_revenue_1 = reg_revenue_1.reset_index()[[
    'i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 'revenue_id', 2011, 2012, 2013,
    2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021
]].rename(columns={
    'revenue_id': 'index'
}).set_index(['i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 'index']).sort_index()

# Tax and non-tax revenues:
reg_revenue_2 = revenue_fin.query(
    'r1 == 1 & 1 <= r2 <= 9 & r3 == 0 & year != 2011 & region_eng == "russia"'
)[['year', 'revenue']].groupby(by='year').sum()
reg_revenue_2 = reg_revenue_2.rename(columns={'revenue': 'reg_tax_revenue'})
reg_revenue_2['reg_nontax_revenue'] = revenue_fin.query(
    'r1 == 1 & 10 <= r2 <= 19 & r3 == 0 & year != 2011 & region_eng == "russia"'
)[['year', 'revenue']].groupby(by='year').sum()
reg_revenue_2.loc[2011, 'reg_tax_revenue'] = revenue_fin.query(
    'r1 == 1 & 1 <= r2 <= 9 & r3 == 0 & year == 2011 & region_eng != "russia"'
)['revenue'].sum()
reg_revenue_2.loc[2011, 'reg_nontax_revenue'] = revenue_fin.query(
    'r1 == 1 & 10 <= r2 <= 19 & r3 == 0 & year == 2011 & region_eng != "russia"'
)['revenue'].sum()
reg_revenue_2 = reg_revenue_2.sort_index()
reg_revenue_2 = reg_revenue_2.T
reg_revenue_2['i2'] = 1
reg_revenue_2['i3'] = 1
reg_revenue_2['r1'] = 1
reg_revenue_2['r2'] = [1, 2]
reg_revenue_2['r3'] = 0
reg_revenue_2['r4'] = 0
reg_revenue_2['r5'] = 0
reg_revenue_2 = reg_revenue_2.reset_index()[[
    'i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 'index', 2011, 2012, 2013, 2014,
    2015, 2016, 2017, 2018, 2019, 2020, 2021
]].set_index(['i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 'index']).sort_index()

# Revenue subgroups:
reg_revenue_3 = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 == 0 & region_eng == "russia"')[[
        'year', 'revenue_type_eng', 'revenue', 'r1', 'r2', 'r3', 'r4'
    ]].dropna().pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
                            columns='year',
                            values='revenue')
reg_revenue_3[2011] = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 == 0 & region_eng != "russia" & year == 2011')[[
        'year', 'region_eng', 'revenue_type_eng', 'revenue', 'r1', 'r2', 'r3',
        'r4'
    ]].dropna().pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
                            columns='year',
                            values='revenue',
                            aggfunc='sum')[2011]
reg_revenue_3 = reg_revenue_3.reset_index()

# Revenue items:
reg_revenue_4 = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 == 0 & region_eng == "russia"')[[
        'year', 'revenue_type_eng', 'revenue', 'r1', 'r2', 'r3', 'r4'
    ]].dropna().pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
                            columns='year',
                            values='revenue')
reg_revenue_4[2011] = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 == 0 & region_eng != "russia" & year == 2011'
)[[
    'year', 'region_eng', 'revenue_type_eng', 'revenue', 'r1', 'r2', 'r3', 'r4'
]].dropna().pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
                        columns='year',
                        values='revenue',
                        aggfunc='sum')[2011]
reg_revenue_4 = reg_revenue_4.reset_index()

# Revenue subitems:
reg_revenue_5 = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 != 0 & region_eng == "russia"')[[
        'year', 'revenue_type_eng', 'revenue', 'r1', 'r2', 'r3', 'r4'
    ]].dropna().pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
                            columns='year',
                            values='revenue')
reg_revenue_5[2011] = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 != 0 & region_eng != "russia" & year == 2011'
)[[
    'year', 'region_eng', 'revenue_type_eng', 'revenue', 'r1', 'r2', 'r3', 'r4'
]].dropna().pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
                        columns='year',
                        values='revenue',
                        aggfunc='sum')[2011]
reg_revenue_5 = reg_revenue_5.reset_index()

reg_revenue_3 = pd.concat([reg_revenue_3, reg_revenue_4, reg_revenue_5])
reg_revenue_3['i2'] = 1
reg_revenue_3['i3'] = 1
reg_revenue_3 = reg_revenue_3.rename(columns={
    'r2': 'r3',
    'r3': 'r4',
    'r4': 'r5'
})


def set_id(s):
    if 1 <= s['r3'] <= 9:
        return 1
    elif 10 <= s['r3'] <= 19:
        return 2


reg_revenue_3['r2'] = reg_revenue_3.apply(set_id, axis=1)
reg_revenue_3 = reg_revenue_3.set_index(
    ['i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5',
     'revenue_type_eng']).sort_index()
reg_revenue_3 = reg_revenue_3[[
    2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021
]]

# Taxes to the federal center: total and by subgroups/items/subitems:
reg_revenue_6 = revenue_fin.query(
    'r1 != 0 & tax_to_fed == tax_to_fed & tax_to_fed != 0 & region_eng == "russia"'
)[['year', 'revenue_type_eng', 'tax_to_fed', 'r1', 'r2', 'r3', 'r4'
   ]].dropna().pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
                           columns='year',
                           values='tax_to_fed').reset_index()
reg_revenue_6['i2'] = 1
reg_revenue_6['i3'] = 1
reg_revenue_6['r1'] = 3
reg_revenue_6 = reg_revenue_6.rename(columns={
    'r2': 'r3',
    'r3': 'r4',
    'r4': 'r5'
})
reg_revenue_6['r2'] = 1
reg_revenue_6 = reg_revenue_6.set_index(
    ['i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5',
     'revenue_type_eng']).sort_index()
reg_revenue_6 = reg_revenue_6[[
    2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021
]]

# Concatinating all the data:
reg_revenue = pd.concat(
    [reg_revenue_1, reg_revenue_2, reg_revenue_3, reg_revenue_6])

# REGIONS/SPENDING

# Total regional spendings
reg_spending_1 = spending.query('s1 == 0 & region_eng == "russia"')[[
    'year', 'spending'
]].reset_index(drop=True).set_index('year')
reg_spending_1 = reg_spending_1.rename(columns={'spending': 'reg_spending'})
reg_spending_1.loc[2011, 'reg_spending'] = spending.query(
    'year == 2011 & s1 == 0')['spending'].sum()
reg_spending_1 = reg_spending_1.sort_index().T
reg_spending_1['i2'] = 1  # 1 for regions 2 for federal budget
reg_spending_1['i3'] = 2  # 1 for revenues 2 for spending
reg_spending_1['s1'] = 0  # revenue section
reg_spending_1['s2'] = 0  # revenue subsection
reg_spending_1 = reg_spending_1.reset_index().set_index(
    ['i2', 'i3', 's1', 's2', 'index'])

# Regional spendings - sections
reg_spending_2 = spending.query(
    'region_eng == "russia" & 1 <= s1 <= 14 & s2 == 0 & spending_id_2 == 0')[[
        'year', 'spending_type_eng', 'spending', 's1', 's2'
    ]].pivot_table(index=['s1', 's2', 'spending_type_eng'],
                   columns='year',
                   values='spending')
reg_spending_2[2011] = spending.query(
    'region_eng != "russia" & 1 <= s1 <= 14 & s2 == 0 & spending_id_2 == 0 & year == 2011'
)[['year', 'spending_type_eng', 'spending', 's1',
   's2']].pivot_table(index=['s1', 's2', 'spending_type_eng'],
                      columns='year',
                      values='spending',
                      aggfunc='sum')
reg_spending_2 = reg_spending_2.reset_index()

# Regional spendings - subsections
reg_spending_3 = spending.query(
    'region_eng == "russia" & 1 <= s1 <= 14 & s2 != 0 & spending_id_2 == 0')[[
        'year', 'spending_type_eng', 'spending', 's1', 's2'
    ]].pivot_table(index=['s1', 's2', 'spending_type_eng'],
                   columns='year',
                   values='spending')
reg_spending_3[2011] = spending.query(
    'region_eng != "russia" & 1 <= s1 <= 14 & s2 != 0 & spending_id_2 == 0 & year == 2011'
)[['year', 'spending_type_eng', 'spending', 's1',
   's2']].pivot_table(index=['s1', 's2', 'spending_type_eng'],
                      columns='year',
                      values='spending',
                      aggfunc='sum')
reg_spending_3 = reg_spending_3.reset_index()

reg_spending_2 = pd.concat([reg_spending_2, reg_spending_3])

reg_spending_2['i2'] = 1
reg_spending_2['i3'] = 2
reg_spending_2 = reg_spending_2.set_index([
    'i2', 'i3', 's1', 's2', 'spending_type_eng'
])[[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]]

reg_spending = pd.concat([reg_spending_1, reg_spending_2])

# FEDERAL/REVENUE

# Total federal revenues:
fed_revenue_1 = fed_rev.query('r1 == 1 & r2 == 0')[['year', 'revenue'
                                                    ]].set_index('year')
fed_revenue_1 = fed_revenue_1.rename(columns={'revenue': 'fed_revenue'})
fed_revenue_1['fed_tax_revenue'] = fed_rev.query(
    'r1 == 1 & 1 <= r2 <= 9 & r3 == 0')[[
        'year', 'revenue_type_eng', 'revenue'
    ]].pivot_table(index='year', values='revenue', aggfunc='sum')
fed_revenue_1['fed_nontax_revenue'] = fed_rev.query(
    'r1 == 1 & 10 <= r2 <= 19 & r3 == 0')[[
        'year', 'revenue_type_eng', 'revenue'
    ]].pivot_table(index='year', values='revenue', aggfunc='sum')
fed_revenue_1 = fed_revenue_1.T
fed_revenue_1['i2'] = 2
fed_revenue_1['i3'] = 1
fed_revenue_1['r1'] = 1
fed_revenue_1['r2'] = [0, 1, 2]
fed_revenue_1['r3'] = 0
fed_revenue_1['r4'] = 0
fed_revenue_1['r5'] = 0
fed_revenue_1 = fed_revenue_1.reset_index().set_index(
    ['i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 'index'])

# Federal revenues by subgroups/items/subitems:
fed_revenue_2 = fed_rev.dropna().query('r1 == 1 & r2 != 0 & r3 == 0')[[
    'year', 'revenue_type_eng', 'revenue', 'r1', 'r2', 'r3', 'r4'
]].pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
               columns='year',
               values='revenue')
fed_revenue_3 = fed_rev.dropna().query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 == 0')[[
        'year', 'revenue_type_eng', 'revenue', 'r1', 'r2', 'r3', 'r4'
    ]].pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
                   columns='year',
                   values='revenue')
fed_revenue_4 = fed_rev.dropna().query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 != 0')[[
        'year', 'revenue_type_eng', 'revenue', 'r1', 'r2', 'r3', 'r4'
    ]].pivot_table(index=['r1', 'r2', 'r3', 'r4', 'revenue_type_eng'],
                   columns='year',
                   values='revenue')
fed_revenue_2 = pd.concat([fed_revenue_2, fed_revenue_3, fed_revenue_4])

fed_revenue_2['i2'] = 2
fed_revenue_2['i3'] = 1
fed_revenue_2 = fed_revenue_2.reset_index().rename(columns={
    'r2': 'r3',
    'r3': 'r4',
    'r4': 'r5'
})


def set_id(s):
    if 1 <= s['r3'] <= 9:
        return 1
    elif 10 <= s['r3'] <= 19:
        return 2


fed_revenue_2['r2'] = fed_revenue_2.apply(set_id, axis=1)
fed_revenue_2 = fed_revenue_2.set_index(
    ['i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5',
     'revenue_type_eng']).sort_index()
# Concatinating:
fed_revenue = pd.concat([fed_revenue_1, fed_revenue_2])

# FEDERAL/SPENDING

fed_spending_1 = fed_spnd.query('s1 == 0 & spnd_id_1 == 0')[[
    'year', 'spending'
]].set_index('year')
fed_spending_1 = fed_spending_1.rename(columns={'spending': 'fed_spending'})
fed_spending_1 = fed_spending_1.T
fed_spending_1['i2'] = 2  # 1 for regions 2 for federal budget
fed_spending_1['i3'] = 2  # 1 for revenues 2 for spending
fed_spending_1['s1'] = 0  # revenue section
fed_spending_1['s2'] = 0  # revenue subsection
fed_spending_1 = fed_spending_1.reset_index().set_index(
    ['i2', 'i3', 's1', 's2', 'index'])

fed_spending_2 = fed_spnd.query(
    's1 != 0 & s2 == 0 & spnd_id_1 != 0 & spnd_id_3 == 0')[[
        'year', 'spending_type_eng', 'spending', 's1', 's2'
    ]].pivot_table(index=['s1', 's2', 'spending_type_eng'],
                   columns='year',
                   values='spending',
                   aggfunc='sum')
fed_spending_3 = fed_spnd.query(
    's1 != 0 & s2 != 0 & spnd_id_1 != 0 & spnd_id_3 == 0')[[
        'year', 'spending_type_eng', 'spending', 's1', 's2'
    ]].pivot_table(index=['s1', 's2', 'spending_type_eng'],
                   columns='year',
                   values='spending',
                   aggfunc='sum')
fed_spending_2 = pd.concat([fed_spending_2, fed_spending_3])

fed_spending_2['i2'] = 2
fed_spending_2['i3'] = 2
fed_spending_2 = fed_spending_2.reset_index().set_index(
    ['i2', 'i3', 's1', 's2', 'spending_type_eng'])

fed_spending = pd.concat([fed_spending_1, fed_spending_2])

df_russia = pd.concat([
    reg_revenue.reset_index(),
    reg_spending.reset_index(),
    fed_revenue.reset_index(),
    fed_spending.reset_index()
],
                      axis=0).set_index([
                          'i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 's1', 's2',
                          'index'
                      ]).sort_index().reset_index()
df_russia = df_russia.fillna(0)
df_russia[['r1', 'r2', 'r3', 'r4', 'r5', 's1',
           's2']] = df_russia[['r1', 'r2', 'r3', 'r4', 'r5', 's1',
                               's2']].astype('int')

add_data = revenue_fin.query('region_eng == "russia"')[[
    'year', 'population', 'real_income', 'income_per_cap', 'poverty', 'rub_usd'
]].groupby(by='year').min()
add_data.loc[2011, 'population'] = regions.query(
    'region_eng == "Russia"')[2011].iloc[0]
add_data.loc[2011, 'real_income'] = real_income.query(
    'region_eng == "Russia"')['2011'].iloc[0]
add_data.loc[2011, 'income_per_cap'] = income_percap.query(
    'region_eng == "Russia"')['2011'].iloc[0]
add_data.loc[2011,
             'poverty'] = poverty.query('region_eng == "Russia"')[2011].iloc[0]
add_data.loc[2011, 'rub_usd'] = rubusd.iloc[0, 0]
add_data = add_data.T

add_data['i3'] = [5, 6, 7, 8, 9]
add_data[['i2', 'r1', 'r2', 'r3', 'r4', 'r5', 's1', 's2']] = 0
add_data = add_data[[
    'i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 's1', 's2', 2011, 2012, 2013,
    2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021
]]

df_russia = pd.concat([df_russia, add_data.reset_index()], axis=0)
df_russia = df_russia.reset_index(drop=True)

df_russia['i1'] = 2
df_russia['region_eng'] = 'russia'
df_russia = df_russia[[
    'i1', 'i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 's1', 's2', 'index',
    'region_eng', 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
    2021
]]

# REGIONS DATA *************************************************************************************

# REVENUE

reg_revenue_1 = revenue_fin.query(
    '(r1 in (0,1) & r2 == 0 & region_eng != "russia") | (r1 == 2 & r2 == 2 & r3 == 0 & region_eng != "russia")'
)[['year', 'revenue_id', 'region_eng', 'revenue', 'tax_to_fed']]
reg_revenue_1['revenue'] = reg_revenue_1['revenue'].fillna(
    reg_revenue_1['tax_to_fed'])
reg_revenue_1 = reg_revenue_1.drop('tax_to_fed', axis=1)
reg_revenue_1 = reg_revenue_1.reset_index(drop=True).pivot_table(
    index=['revenue_id',
           'region_eng'], columns='year', values='revenue').reset_index()
reg_revenue_1['revenue_id'] = reg_revenue_1['revenue_id'].replace(
    '00000000000000000', 'tax_to_fed', regex=True)
reg_revenue_1['revenue_id'] = reg_revenue_1['revenue_id'].replace(
    '10000000000000000', 'reg_own_revenue', regex=True)
reg_revenue_1['revenue_id'] = reg_revenue_1['revenue_id'].replace(
    '20200000000000000', 'transfers_to_reg', regex=True)
reg_revenue_1['i3'] = 1


def set_id(s):
    if s['revenue_id'] == 'reg_own_revenue':
        return 1
    elif s['revenue_id'] == 'transfers_to_reg':
        return 2
    elif s['revenue_id'] == 'tax_to_fed':
        return 3


reg_revenue_1['r1'] = reg_revenue_1.apply(set_id, axis=1)
reg_revenue_1['r3'] = 0
reg_revenue_1['r4'] = 0
reg_revenue_1['r5'] = 0
reg_revenue_1 = reg_revenue_1.set_index(
    ['i3', 'r1', 'r3', 'r4', 'r5', 'revenue_id', 'region_eng'])

reg_revenue_2 = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 == 0 & region_eng != "russia"')[[
        'year', 'region_eng', 'r2', 'r3', 'r4', 'revenue_type_eng', 'revenue'
    ]].dropna()
reg_revenue_2['i3'] = 1
reg_revenue_2['r1'] = 1
reg_revenue_2 = reg_revenue_2.rename(columns={
    'r2': 'r3',
    'r3': 'r4',
    'r4': 'r5',
    'revenue_type_eng': 'index'
}).pivot_table(index=['i3', 'r1', 'r3', 'r4', 'r5', 'index', 'region_eng'],
               columns='year',
               values='revenue')

reg_revenue_3 = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 == 0 & region_eng != "russia"')[[
        'year', 'region_eng', 'r2', 'r3', 'r4', 'revenue_type_eng',
        'tax_to_fed'
    ]].dropna()
reg_revenue_3['i3'] = 1
reg_revenue_3['r1'] = 3
reg_revenue_3 = reg_revenue_3.rename(columns={
    'r2': 'r3',
    'r3': 'r4',
    'r4': 'r5',
    'revenue_type_eng': 'index'
}).pivot_table(index=['i3', 'r1', 'r3', 'r4', 'r5', 'index', 'region_eng'],
               columns='year',
               values='tax_to_fed')

reg_revenue_4 = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 == 0 & region_eng != "russia"')[[
        'year', 'region_eng', 'r2', 'r3', 'r4', 'revenue_type_eng', 'revenue'
    ]].dropna()
reg_revenue_4['i3'] = 1
reg_revenue_4['r1'] = 1
reg_revenue_4 = reg_revenue_4.rename(columns={
    'r2': 'r3',
    'r3': 'r4',
    'r4': 'r5',
    'revenue_type_eng': 'index'
}).pivot_table(index=['i3', 'r1', 'r3', 'r4', 'r5', 'index', 'region_eng'],
               columns='year',
               values='revenue')

reg_revenue_5 = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 == 0 & region_eng != "russia"')[[
        'year', 'region_eng', 'r2', 'r3', 'r4', 'revenue_type_eng',
        'tax_to_fed'
    ]].dropna()
reg_revenue_5['i3'] = 1
reg_revenue_5['r1'] = 3
reg_revenue_5 = reg_revenue_5.rename(columns={
    'r2': 'r3',
    'r3': 'r4',
    'r4': 'r5',
    'revenue_type_eng': 'index'
}).pivot_table(index=['i3', 'r1', 'r3', 'r4', 'r5', 'index', 'region_eng'],
               columns='year',
               values='tax_to_fed')

reg_revenue_6 = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 != 0 & region_eng != "russia"')[[
        'year', 'region_eng', 'r2', 'r3', 'r4', 'revenue_type_eng', 'revenue'
    ]].dropna()
reg_revenue_6['i3'] = 1
reg_revenue_6['r1'] = 1
reg_revenue_6 = reg_revenue_6.rename(columns={
    'r2': 'r3',
    'r3': 'r4',
    'r4': 'r5',
    'revenue_type_eng': 'index'
}).pivot_table(index=['i3', 'r1', 'r3', 'r4', 'r5', 'index', 'region_eng'],
               columns='year',
               values='revenue')

reg_revenue_7 = revenue_fin.query(
    'r1 == 1 & r2 != 0 & r3 != 0 & r4 != 0 & region_eng != "russia"')[[
        'year', 'region_eng', 'r2', 'r3', 'r4', 'revenue_type_eng',
        'tax_to_fed'
    ]].dropna()
reg_revenue_7['i3'] = 1
reg_revenue_7['r1'] = 3
reg_revenue_7 = reg_revenue_7.rename(columns={
    'r2': 'r3',
    'r3': 'r4',
    'r4': 'r5',
    'revenue_type_eng': 'index'
}).pivot_table(index=['i3', 'r1', 'r3', 'r4', 'r5', 'index', 'region_eng'],
               columns='year',
               values='tax_to_fed')

reg_revenue = pd.concat([
    reg_revenue_1, reg_revenue_2, reg_revenue_3, reg_revenue_4, reg_revenue_5,
    reg_revenue_6, reg_revenue_7
])
reg_revenue = reg_revenue.sort_index().reset_index()
reg_revenue = reg_revenue.rename(columns={'revenue_id': 'index'})

# SPENDING

reg_spending_1 = spending.query('s1 == 0 & region_eng != "russia"')[[
    'year', 'region_eng', 'spending'
]].reset_index(drop=True).pivot_table(index='region_eng',
                                      columns='year',
                                      values='spending')
reg_spending_1['index'] = 'reg_spending'
reg_spending_1['i3'] = 2
reg_spending_1['s1'] = 0
reg_spending_1['s2'] = 0

reg_spending_1 = reg_spending_1.reset_index().set_index(
    ['i3', 's1', 's2', 'index', 'region_eng'])

reg_spending_2 = spending.query('s1 != 0 & s2 == 0 & region_eng != "russia"')[[
    'year', 'region_eng', 's1', 's2', 'spending_type_eng', 'spending'
]].reset_index(drop=True).pivot_table(
    index=['s1', 's2', 'spending_type_eng', 'region_eng'],
    columns='year',
    values='spending').reset_index()

reg_spending_3 = spending.query(
    's1 != 0 & s2 != 0 & spending_id_2 == 0 & region_eng != "russia"')[[
        'year', 'region_eng', 's1', 's2', 'spending_type_eng', 'spending'
    ]].reset_index(drop=True).pivot_table(
        index=['s1', 's2', 'spending_type_eng', 'region_eng'],
        columns='year',
        values='spending').reset_index()

reg_spending_3 = reg_spending_3[reg_spending_3['spending_type_eng'].notna()]
reg_spending_2 = pd.concat([reg_spending_2, reg_spending_3])
reg_spending_2['i3'] = 2
reg_spending_2 = reg_spending_2.rename(columns={
    'spending_type_eng': 'index'
}).set_index(['i3', 's1', 's2', 'index', 'region_eng'])

reg_spending = pd.concat([reg_spending_1, reg_spending_2])

# CONCATINATING

df_regions = pd.concat([reg_revenue, reg_spending.reset_index()],
                       axis=0).set_index(
                           ['i3', 'r1', 'r3', 'r4', 'r5', 's1', 's2',
                            'index']).sort_index().reset_index()
df_regions = df_regions.fillna(0)
df_regions[['r1', 'r3', 'r4', 'r5', 's1',
            's2']] = df_regions[['r1', 'r3', 'r4', 'r5', 's1',
                                 's2']].astype('int')

add_data = revenue_fin.query('region_eng != "russia"')[[
    'year', 'region_eng', 'population', 'real_income', 'income_per_cap',
    'poverty', 'rub_usd'
]].groupby(by=['year', 'region_eng']).min()
add_data_pivot = pd.melt(
    add_data.reset_index(),
    id_vars=['year', 'region_eng'],
    value_vars=[
        'population', 'real_income', 'income_per_cap', 'poverty', 'rub_usd'
    ],
    var_name='index').pivot_table(index=['index', 'region_eng'],
                                  columns='year',
                                  values='value').reset_index()


def set_id(s):
    if s['index'] == "population":
        return 5
    elif s['index'] == "real_income":
        return 6
    elif s['index'] == "income_per_cap":
        return 7
    elif s['index'] == "poverty":
        return 8
    elif s['index'] == "rub_usd":
        return 9


add_data_pivot['i3'] = add_data_pivot.apply(set_id, axis=1)
add_data_pivot[['r1', 'r3', 'r4', 'r5', 's1', 's2']] = 0

df_regions = pd.concat([df_regions, add_data_pivot], axis=0)
df_regions = df_regions.reset_index(drop=True)

df_regions['i1'] = 1
df_regions['i2'] = 1


def set_id(s):
    if 1 <= s['r3'] <= 9:
        return 1
    elif 10 <= s['r3'] <= 19:
        return 2
    else:
        return 0


df_regions['r2'] = df_regions.apply(set_id, axis=1)

df_regions = df_regions[[
    'i1', 'i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 's1', 's2', 'index',
    'region_eng', 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
    2021
]]

budget_data = pd.concat([df_russia, df_regions], axis=0)

budget_data = pd.melt(budget_data,
                      id_vars=[
                          'i1', 'i2', 'i3', 'r1', 'r2', 'r3', 'r4', 'r5', 's1',
                          's2', 'index', 'region_eng'
                      ],
                      value_vars=[
                          2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
                          2020, 2021
                      ],
                      var_name='year')

In [35]:
budget_data

,i1,i2,i3,r1,r2,r3,r4,r5,s1,s2,index,region_eng,year,value
0,2,1,1,1,0,0,0,0,0,0,reg_own_revenue,russia,2011,5.825363e+12
1,2,1,1,1,1,0,0,0,0,0,reg_tax_revenue,russia,2011,5.280061e+12
2,2,1,1,1,1,1,0,0,0,0,"profit taxes, revenues",russia,2011,3.922583e+12
3,2,1,1,1,1,1,1,0,0,0,corporate income tax full,russia,2011,1.928099e+12
4,2,1,1,1,1,1,1,10,0,0,corporate income tax,russia,2011,1.926076e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161222,1,1,9,0,0,0,0,0,0,0,rub_usd,vologda oblast,2021,7.365410e+01
161223,1,1,9,0,0,0,0,0,0,0,rub_usd,voronezh oblast,2021,7.365410e+01
161224,1,1,9,0,0,0,0,0,0,0,rub_usd,yamalo-nenets autonomous okrug,2021,7.365410e+01
161225,1,1,9,0,0,0,0,0,0,0,rub_usd,yaroslavl oblast,2021,7.365410e+01


In [36]:
budget_data.to_csv('final_data/russian_budget_data.csv')

In [37]:
time2 = time()
print('Total notebook execution time: ', round((time2-time1)/60), ' minutes')

Total notebook execution time:  18  minutes
